# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 2924, 2, 1263, 2, 1578, 72, 643, 7, 4, 1255, 19, 1265, 2390, 11, 4, 217, 7, 4, 1002, 5, 1024, 2, 9, 4, 430, 45, 6, 2, 22, 18, 2714, 13, 482, 21, 12, 82, 2, 2, 44, 4, 2, 7, 992, 349, 76, 9, 93, 7, 1619, 2, 11, 63, 349, 193, 23, 668, 555, 4, 20, 2679, 23, 35, 2, 132, 37, 494, 2724, 8, 30, 2, 5, 1487, 34, 6, 2, 232, 29, 2, 4, 1547, 209, 1092, 29, 1915, 8, 30, 623, 83, 4, 2, 34, 4, 1909, 5, 15, 11, 192, 571, 95, 29, 716, 4, 1909, 27, 436, 113, 65, 137, 112, 2, 8, 3419, 1789, 93, 8, 2, 18, 6, 810, 7, 58, 2, 2, 14, 9, 51, 4, 992, 132, 494, 5, 2, 4, 992, 132, 9, 4739, 4, 2, 532, 1459, 17, 6, 2, 132, 4, 2, 44, 4, 992, 1601, 113, 63, 1470, 2183, 26, 2, 21, 54, 29, 517, 2, 154, 1619, 2, 5, 905, 495, 4, 20, 461, 3809, 12, 461, 6, 1415, 7, 2240, 18, 2, 2, 37, 64, 181, 8, 2, 6, 2040, 217, 11, 1619, 926, 806, 13, 191, 2198]
--- Label ---
0


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'midnight', 'and', 'sitting', 'and', 'inspired', 'we', 'cool', 'br', 'of', 'shoot', 'film', 'pictures', 'louis', 'this', 'of', 'almost', 'br', 'of', 'outside', 'to', 'attempts', 'and', 'it', 'of', 'worse', 'if', 'is', 'and', 'you', 'but', 'understood', 'was', 'lead', 'not', 'that', 'other', 'and', 'and', 'has', 'of', 'and', 'br', 'dumb', 'budget', 'get', 'it', 'way', 'br', 'location', 'and', 'this', 'really', 'budget', 'long', 'are', 'power', 'god', 'of', 'on', 'quest', 'are', 'so', 'and', 'say', 'like', 'tries', 'dress', 'in', 'at', 'and', 'to', 'gorgeous', 'who', 'is', 'and', 'feel', 'all', 'and', 'of', 'hospital', 'comedy', 'd', 'all', 'vs', 'in', 'at', 'released', 'first', 'of', 'and', 'who', 'of', 'broken', 'to', 'for', 'this', 'enough', 'involved', 'them', 'all', 'local', 'of', 'broken', 'be', 'problem', 'acting', 'their', 'go', 'never', 'and', 'in', 'dawn', 'jr', 'way', 'in', 'and'

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [4]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train, X_test = sequence.pad_sequences(X_train, max_words), sequence.pad_sequences(X_test, max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [16]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32

# TODO: Design your model
model = Sequential([
    Embedding(vocabulary_size, embedding_size, input_length=max_words),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [18]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [19]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 32
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)

# TODO: Train your model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs)

Epoch 1/3
25000/25000 [==============================] - 298s 12ms/step - loss: 0.4402 - acc: 0.7958
Epoch 2/3
25000/25000 [==============================] - 298s 12ms/step - loss: 0.3003 - acc: 0.8765
Epoch 3/3
25000/25000 [==============================] - 296s 12ms/step - loss: 0.2439 - acc: 0.9047


In [22]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join('./', model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [23]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.86708


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!